# Installing graph-tool (https://graph-tool.skewed.de)

---


To import a library that's not in Colaboratory by default, you can use `!apt-get install`.

But since graph-tool is not in the official repository, we need to add it to the list.

In [1]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

Executing: /tmp/apt-key-gpghome.WwVYyeIbic/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://downloads.skewed.de/apt bionic InRelease [7,540 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidi

In [2]:
#python3-cairo from Ubuntu's reposity is linked with a different python version; we need to improvise
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  python3-cairo*
0 upgraded, 0 newly installed, 1 to remove and 66 not upgraded.
After this operation, 257 kB disk space will be freed.
(Reading database ... 159433 files and directories currently installed.)
Removing python3-cairo:amd64 (1.16.2-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libcairo-script-interpreter2 lib

# Repository of weighted graphs: https://networks.skewed.de/?tags=Weighted

In [64]:
import pandas as pd
import graph_tool.all as gt
from google.colab import files

from google.colab import drive

import os.path
from os import path

def export_all_weightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive="weightedGraphs/"):
   #Exports in ERODE CSV format all weighted graphs in the list model_names. 
   #The CSV file is either stored on google drive, or downloaded, as required by the parameters (Google drive has precedence on downloading)
   #If the google drive option is chosen, the files will be stored in folder weightedGraphs, or different paths provided in parameter path_g_drive
   #  This should be a relative path to a folder (ending with a slash). E.g., if you pass "weightedGraphs/", we will store files in it in MyDrive/weightedGraphs/

  print('Processing and exporting models')
  print("",model_names)

  #Prepare gdrive
  gdrive_path="/content/drive/MyDrive/"+path_g_drive
  if store_on_gdrive:
    drive.mount('/content/drive')
    if not path.exists(gdrive_path):
      os.mkdir(gdrive_path)

  #Export required weighted graps
  for model_name in model_names:
    export_weightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path)

  #Close gdrive
  if store_on_gdrive:
    drive.flush_and_unmount()

  print('All computations completed')

def export_weightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path):
  #load model
  g = gt.collection.ns[model_name]

  #store unweighted edges in a dataframe, and add weights
  df = pd.DataFrame(data=g.get_edges(), columns=["source", "target"])
  df['weight']=g.edge_properties['weight'].get_array()
  
  n_nodes=g.num_vertices()
  df0 = pd.DataFrame([[n_nodes,n_nodes,n_nodes]], columns=["source", "target",'weight'])
  df=pd.concat([df0, df])
  df.reset_index(drop=True, inplace=True)
  print(df)


  #compute file name of csv file
  csv_file_name=model_name+'_A.csv'
  if store_on_gdrive:
    csv_file_name=gdrive_path+model_name+'.csv'
  print(" ",csv_file_name),"..."

  print("  ","Graph processed, and data structure created")

  #create csv file and download if required
  df.to_csv(csv_file_name,index=False,header=False)
  if download_file and not store_on_gdrive:
    files.download(csv_file_name)

  print("  ","CSV file exported")
  
#Choose the models to consider
# Just https://networks.skewed.de/net/blumenau_drug
model_names=["blumenau_drug"]

#All weighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Weighted after clicking on 'Nodes'
#model_names=["packet_delays","ambassador","rhesus_monkey","kangaroo","sp_baboons","bison","cattle","moreno_sheep","college_freshmen","hens","freshmen","windsurfers","macaques","train_terrorists","highschool","blumenau_drug","lesmis","baseball","sp_office","game_thrones","moviegalaxies","foodweb_baywet","plant_pol_vazquez","dom","cintestinalis","faculty_hiring","fao_trade","residence_hall","un_migrations","sp_primary_school","celegansneural","sp_high_school","london_transport","sp_colocation","webkb","celegans_2019","messal_shale","plant_pol_kato","copenhagen","product_space","unicodelang","budapest_connectome","new_zealand_collab","netscience","bible_nouns","add_health","openflights","bitcoin_alpha","bitcoin_trust","advogato","foldoc","physics_collab","escorts","fly_hemibrain","arxiv_collab","us_agencies","eu_procurements_alt","jester","libimseti","wiki_article_words","bookcrossing","twitter_higgs","eu_procurements","epinions","yahoo_song","mag_history_coauthor","mag_geology_coauthor","amazon_ratings","dblp_simplices","bag_of_words","microsoft_concept"]

#Set flags on where to store the obtained CSV files
download_file=True
store_on_gdrive=True
path_g_drive="weightedGraphs/"
export_all_weightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive)

Processing and exporting models
 ['blumenau_drug']
Mounted at /content/drive
  /content/drive/MyDrive/weightedGraphs/blumenau_drug.csv
   Graph processes and data structure created
   Exported
All computations completed


# If interested in drawing

In [ ]:
# from graph_tool.all import *
# g = collection.data["celegansneural"]
# state = minimize_nested_blockmodel_dl(g)
# state = minimize_nested_blockmodel_dl(g)
# state.draw()